<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

# Best video games from 1952 to 2029

<img src="images/he-arc.png"  width="200" style="background-color:white ; padding: 10px">

### MAS-RAD CAS-IDD Data processing and visualization with Python

#### Teacher : Benoit Le Callennec
#### Authors : Ayman Akram & Rafael Teixeira

All rights reserved
</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Initialisation of the project

Before running the notebook, make sure the dependencies have been installed. See the [README file](README.md) for instructions

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px;">

## Table of contents

- [Introduction](#introduction)
- [Data import and cleaning](#data-important-and-cleaning)
- [Data analysis](#data-analysis)
  - [Number of games per genre](#number-of-games-per-genre)
  - [Most played games](#most-played-games)
  - [Success per genre over the years](#success-per-genre-over-the-years)
- [Summary](#summary)
- [Limitations and perspectives](#limitations-and-perspectives)

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Introduction

We want to statistically analyze a dataset of video games, answering questions about their characteristics, popularity and evolution over time. We also want to extract key information about their genres, popularity and various correlations.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Data import and cleaning
Import the data from kaggle [(link to data)](https://www.kaggle.com/datasets/matheusfonsecachaves/popular-video-games/data?select=backloggd_games.csv).

If an error succeeds during data download, a fallback file is used. This file contains a previous version of the data and so, it may not be up to date.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Load data and store it in pandas DataFrame
</div>

In [ ]:
# Import all dependencies
import os
import re
import kagglehub
import pandas as pd
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
import IPython

In [ ]:
file_path = ""
FILE_NAME = "backloggd_games.csv"
FALLBACK_PATH = "Datasets/"

try:
    # Download last version of dataset from kaggle
    path = kagglehub.dataset_download("matheusfonsecachaves/popular-video-games")
    print("Last version of file successfully loaded from kaggle.")
    file_path = path + "\\" + FILE_NAME

except:
    # If an exception is raised when downloading last version of dataset, the locally saved version is used. It may no be up to date
    print("Last version could not be successfully loaded from kaggle.\nThe locally saved dataset will be used but it may not be up to date.")
    file_path = FALLBACK_PATH + FILE_NAME

print("Path to dataset file:", file_path)

if os.path.exists(file_path):
    df = pd.read_csv(file_path)

else:
    print("The file does not exist. Check the path or place the file in the correct location.")

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Data basic verifications

</div>

In [ ]:
# Data basic verification : shape of dataFrame, columns informations and head
print("DataFrame shape : " + str(df.shape)) # row and columns
print("DataFrame info :")
df.info()
print("DataFrame head :")
print(df.head(6)) # Preview

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Null data and duplicates analysis

</div>

In [ ]:
# Null verification
df.isnull().sum()

In [ ]:
df.duplicated().sum()

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

There is no duplicate entry in the dataset.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Convert date column values to numpy date

The Release_Date column contains dates with the following format.

For example 2 February 2023 will be Feb 02, 2023.

To work with those dates, they must be converted to a more convenient format such as numpy date.

When analysing the data of this column, some undefined dates can be found. Their value is 'TBD' and we choose to turn it to Nan so it is easier to remove when needed.

The column "Release_Date" values are replaced by their equivalent numpy date value or by numpy.nan if they do not match the defined date pattern.

</div>

In [ ]:
# Regex values for date string
DATE_REGEX = re.compile(r'[A-Z]{3} \d\d, \d{4}', re.I)
MONTH_REGEX = re.compile(r'[A-Z]{3}', re.I)
DAY_REGEX = re.compile(r'\d{2}(?=,)')
YEAR_REGEX = re.compile(r'(\d{4}$)')

# Dictionnary to convert month string to month number
MONTH_DICT = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}

def mapMonth(dateStr):
    # Converts month string to number
    global DATE_REGEX
    matches = re.match(DATE_REGEX, dateStr)
    if matches:
        return convertToNumpyDate(matches[0])
    else:
        return np.nan

def convertToNumpyDate(dateStr):
    # Converts date string to numpy date
    global MONTH_REGEX, DAY_REGEX, YEAR_REGEX, MONTH_DICT
    # Find month :
    monthStr = re.search(MONTH_REGEX, dateStr)[0].lower()
    month = MONTH_DICT[monthStr]
    day = int(re.search(DAY_REGEX, dateStr)[0])
    year = int(re.search(YEAR_REGEX, dateStr)[0])
    return np.datetime64(f"{year}-{month:02d}-{day:02d}")


df['Release_date_converted'] = df['Release_Date'].map(mapMonth)
df.info()

print("")
print("Before conversion : " + df['Release_Date'][0])
print("After conversion : " + str(df['Release_date_converted'][0]))

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Convert Plays from string to number
The data in 'Plays' column is at string format of type 2K, 2M, etc.

We convert those values to number, removing the suffix K or M and multiplying by the right multiplier (1'000 or 1'000'000).

</div>

In [ ]:
print("Before conversion : " + str(df['Plays'][0]))

def parse_play_count(x):
    # Convert K and M abbreviations to integers
    x = str(x).replace(',', '').strip().upper()
    if x.endswith('K'):
        return int(float(x[:-1]) * 1_000)
    elif x.endswith('M'):
        return int(float(x[:-1]) * 1_000_000)
    elif x.isdigit():
        return int(x)
    else:
        return None  # to invalid values

df['Plays'] = df['Plays'].apply(parse_play_count)
print("After conversion : " + str(df['Plays'][0]))

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Drop unused colums

</div>

In [ ]:
try :
    df.drop(["Developers", "Summary", "Platforms", "Playing", "Backlogs", "Wishlist", "Lists"], axis=1, inplace=True)
except Exception as e:
    print("An error was raised. It could be because columns have already been erased.")
    print(e)
df.shape

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Data analysis

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Number of games per genre
This analysis explores the distribution of video games across different genres, using the "Genres" column.
Since a game can belong to multiple genres, the data is cleaned and split accordingly to identify the most frequently occurring genres.
This helps highlight industry trends in game development (e.g., Adventure, Indie, RPG, etc.).

</div>

In [ ]:
# Remove missing values, convert to string, and clean characters like [], '
all_genres = df['Genres'].dropna().astype(str).str.replace(r'[\[\]\']', '', regex=True)

# Split genres by comma, explode into rows, and strip whitespace
all_genres = all_genres.str.split(',').explode().str.strip()

# Remove empty genres
all_genres = all_genres[all_genres != ""]

# Count occurrences
genre_counts = all_genres.value_counts()
print("Number of games per genre :\n", genre_counts.head(10))

# Widget to select how many genres to display
top_n_slider1 = ipw.IntSlider(value=10, min=1, max=23, step=1, description='Top Game Genres:')

# Graphical
def plot_top_n1(n):
    plt.figure(figsize=(8, 4))
    genre_counts.head(n).plot(kind='bar')
    plt.title(f"Top {n} Game Genres")
    plt.xlabel("Genre")
    plt.ylabel("Number of games")
    plt.xticks(rotation=0)
    plt.tight_layout()
    # plt.show()


# Binding and display
IPython.display.display(ipw.VBox([top_n_slider1, ipw.interactive_output(plot_top_n1, {'n': top_n_slider1})]))

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

We see that video games that are of the adventure and indie genre are produced much more than other genres.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Most played games
This section focuses on game popularity, measured via the "Plays" column in combination with "Title".
We then identify the most played games, giving insight into what players engage with the most. Minecraft being the standout leader.

</div>

In [ ]:

# Widget to select how many genres to display
top_n_slider2 = ipw.IntSlider(value=10, min=1, max=100, step=1, description='Top Played Games:')

# Graphical
def plot_top_n2(n):
    top_df = df[['Title', 'Plays']] \
        .sort_values(by='Plays', ascending=False) \
        .head(n)

    plt.figure(figsize=(12, 10))
    plt.bar(top_df['Title'], top_df['Plays'])
    plt.title(f"Top {n} Most played games")
    plt.xlabel("Title of games")
    plt.ylabel("Number of games")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    # plt.show()

# Binding and display
IPython.display.display(ipw.VBox([top_n_slider2, ipw.interactive_output(plot_top_n2, {'n': top_n_slider2})]))


<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

We see that the most played game is Minecraft, it is the one that allows the greatest creativity and the most freedom.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

### Success per genre over the years

To determine the succes of a game, we consider the number of plays it has and its rating.

To calculate a success score, the following formula is used :

success_score = rating^4 * plays^0.5

Using a power 4 on "rating", increases "rating" weight on success score.

On the contrary, a power of 0.5 on "plays", decreases "plays" weight on success score.

</div>

In [ ]:
# Sort by Release date
df.sort_values("Release_date_converted", inplace=True)

# Calculate Success_score for each game
df['Weighted_rating'] = df["Rating"]**4
df['Weighted_number_plays'] = df["Plays"]**0.5
df['Success_score'] = df['Weighted_rating'] * df['Weighted_number_plays']

# Get a copy of the dataframe, drop Nan values and replace
success_score_and_genre_per_date = df[['Release_date_converted','Genres', 'Success_score']].dropna()

genres = df['Genres'].str.replace(r'[\[\]\']', '', regex=True).str.split(',').explode().str.strip().drop_duplicates().sort_values()

comparison_mode_widget = ipw.ToggleButton(value=False, description="Comparison mode")
genre_dropdown = ipw.Dropdown(options=genres, description="Genre 1:")
genre_dropdown_2 = ipw.Dropdown(options=genres, description="Genre 2:")

def getSuccessScoresByGenre(dataframe, genre):
    # Function to filter dataframe keeping only the lines where column 'Genres' contains wanted genre
    filtered = dataframe[dataframe["Genres"].map(lambda x: genre in x)]
    return filtered


def plotChart(comparison_mode, genre1, genre2):
    # Function to plot the chart
    global success_score_and_genre_per_date, genre_dropdown_2

    # Enable/Disable genre2 dropdown based on comparison mode value
    genre_dropdown_2.disabled = not comparison_mode

    # Get filtered values for genre 1 and group by year
    filtered = getSuccessScoresByGenre(success_score_and_genre_per_date, genre1)
    success_score_per_year_genre1 = filtered['Success_score'].groupby(filtered['Release_date_converted'].dt.to_period("Y")).sum()
    # Convert years to int and plot for genre 1
    success_score_per_year_genre1.index = success_score_per_year_genre1.index.astype("str").astype("int64")
    plt.plot(success_score_per_year_genre1.index, success_score_per_year_genre1.values)
    # Change empty genre value to "All genre" to have a more meaningful legend and title
    if not genre1:
        genre1 = "All genre"

    if comparison_mode:
        # Get filtered values for genre 2 and group by year
        filtered = getSuccessScoresByGenre(success_score_and_genre_per_date, genre2)
        success_score_per_year_genre2 = filtered['Success_score'].groupby(filtered['Release_date_converted'].dt.to_period("Y")).sum()
        # Convert years to int and plot for genre 2
        success_score_per_year_genre2.index = success_score_per_year_genre2.index.astype("str").astype("int64")
        plt.plot(success_score_per_year_genre2.index, success_score_per_year_genre2.values)
        # Change empty genre value to "All genre" to have a more meaningful legend and title
        if not genre2:
            genre2 = "All genre"


    # Update chart informations
    if comparison_mode:
        title = "Success of '" + genre1 + "' games vs '" + genre2 + "' games over the years"
    else:
        title = "Success of '" + genre1 + "' games over the years"

    plt.title(title)
    plt.xlabel("Release year")
    plt.ylabel("Success score")
    plt.legend([genre1, genre2])
    plt.tight_layout()

ipw.interact(plotChart, comparison_mode=comparison_mode_widget, genre1=genre_dropdown, genre2=genre_dropdown_2)

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

The above analysis shows that the success score drops after 2020.
This drop can probably be explained with different causes.

The analyzed dataset most recent games were released in 2023.

We can quickly analyze when the 'Plays' value seems to start dropping (see chart below).

We can see that 'Plays' start dropping after 2017. We can make the hypothesis that data of games released after 2017 is not complete or those the games are too recent to be compared to older games.

</div>

In [ ]:
plays_per_year = df['Plays'].groupby(df['Release_date_converted'].dt.to_period("Y")).sum()
plays_per_year.plot(kind="line", marker= 'o')
plt.xlabel("Release year")
plt.ylabel("Number of plays")

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Summary

Using the dataset 'Popular Video Games' from Matheus Fonseca Chaves posted on kaggle.com ([Link](https://www.kaggle.com/datasets/matheusfonsecachaves/popular-video-games/data?select=backloggd_games.csv)), we tried to show interesting facts about video games genres released from 1952 to 2023.

Before the analysis, we cleaned the data from unwanted columns and converted some values to make them usable for analysis :

- Conversion of the 'Number of plays' : Expressed with thousand and million suffixes (e.g. 24K), we converted it to numbers so the values can be user for calculation and chart representation.
- Conversion of 'Release date' : Expressed in a string of type "Feb 25, 2022", we converted it to numpy dates so they can be used to calculate and plot values based on date.

We then analysed and plot data in 3 ways :
- Number of games per genre
- Most played games
- Success per genre over the years


The "number of games per genre" analysis shows which genres have the most released number of games. The results show that Adventure games and Indie games are the most released genres.


The "most played games" analysis sorts the games by 'number of plays'. It shows that Minecraft and The Legend of Zelda : Breath of the Wild are the most played games of all time.


The "success per genre over the years" analysis calculates a "success score" for each game based on rating and number of plays.

The analysis allows the user to show the evolution of a genres success over time and to compare between 2 genres.

This last analysis shows that the "success" globally drops after 2017. We have 2 hypothesis to explain this drop :
1. Recent games have less "plays" than long established games
2. Most recent released games may have incomplete data

Also, there seems to be some "strange" data in the dataset. For example, a game released in 2029 is present (the title pun is based on that entry). Some very old games are also present in the data but some years are empty. We know that the video game industry has really grown after the 80's but if a game released in 1952 is mentioned, we may think that some games from 1952 to 1980 may not be included in the dataset.

</div>

<div style="padding:10px;border-left:10px solid #9999ff; border-radius:5px">

## Limitations and perspectives

This dataset has some limitations.

Ratings are missing for almost 50% of the games making it difficult to base analysis on this property.

The dataset is not automatically updated based, for example, on vendors or distributors data. This means the data is never up to date. It is a state at a specific time. This means that the conclusions are only valid for the period until 2023 (last data update). Games released after 2023 are not mentioned in this dataset and the values of older games are not up to date.


Still, there are some perspectives with this data. We can think about different projects and analysis based on this dataset.

Many examples can come to mind :
- Game finder, based on success, genre, platform, developers, etc.
- Comparison between platforms
- Word analysis based on titles or summaries
- etc.

The dataset could also be cross-referenced with other data to extend the possibilities.

</div>